# Intelligent Agents: Vacuum-cleaner World


## Instructions

Total Points: undergrad 10, graduate students 11

Complete this notebook and submit it. The notebook needs to be a complete project report with 

* your implementation,
* documentation including a short discussion of how your implementation works and your design choices, and
* experimental results (e.g., tables and charts with simulation results) with a short discussion of what they mean. 

Use the provided notebook cells and insert additional code and markdown cells as needed.

## Introduction

In this assignment you will implement a simulator environment for an automatic vacuum cleaner robot, a set of different agent programs, and perform a comparison study for cleaning a single room. Focus on the __cleaning phase__ which starts when the robot is activated and ends when the last dirty square is cleaned. Someone else will take care of the agent program needed to navigate back to the charging station after the room is clean.

## PEAS description of the cleaning phase

__Performance Measure:__ Each action costs 1 energy unit. The performance is measured as the sum of the energy units used to clean the whole room.

__Environment:__ A room with $n \times n$ squares where $n = 5$. Dirt is randomly placed on each square with probability $p = 0.2$. For simplicity, you can assume that the agent knows the size and the layout of the room (i.e., it knows $n$). To starte, the agent is placed on a random square.

__Actuators:__ The agent can `clean` the current square or move to an adjacent square by going `north`, `east`, `south`, or `west`.

__Sensors:__ Four bumper sensors, one for north, east, south, and west; a dirt sensor reporting dirt in the current square.  


## The agent program for a simple randomized agent

The agent program is a function that gets sensor information (the current percepts) as the arguments. The arguments are:

* A dictionary with boolean entries for the for bumper sensors `north`, `east`, `west`, `south`. E.g., if the agent is on the north-west corner, `bumpers` will be `{"north" : True, "east" : False, "south" : False, "west" : True}`.
* The dirt sensor produces a boolean.

The agent returns the chosen action as a string.

Here is an example implementation for the agent program of a simple randomized agent:  

In [1]:
from numpy import random

actions = ["north", "east", "west", "south", "suck"]

def simple_randomized_agent(bumpers, dirty):
    return random.choice(actions)

In [2]:
# define percepts (current location is NW corner and it is dirty)
bumpers = {"north" : True, "east" : False, "south" : False, "west" : True}
dirty = True

# call agent program function with percepts and it returns an action
simple_randomized_agent(bumpers, dirty)

'suck'

__Note:__ This is not a rational intelligent agent. It ignores its sensors and may bump into a wall or not clean a dirty square. You will be asked to implement rational agents below.

## Simple environment example

This simple environment is infinite in size (bumpers are always `False`) and every square is always dirty, even if the agent cleans it. The environment function returns the performance measure which is here the number of cleaned squares (since all squares are constantly dirty, it is the number of `suck` actions by the agent). 

In [3]:
def simple_environment(agent, max_steps, verbose = True):
    num_cleaned = 0
    
    for i in range(max_steps):
        dirty = True
        bumpers = {"north" : False, "south" : False, "west" : False, "east" : False}

        action = agent(bumpers, dirty)
        if (verbose): print("step", i , "- action:", action) 
        
        if (action == "suck"): 
            num_cleaned = num_cleaned + 1
        
    return num_cleaned
        


Do one simulation run with 20 steps.

In [4]:
simple_environment(simple_randomized_agent, max_steps = 20)

step 0 - action: north
step 1 - action: suck
step 2 - action: suck
step 3 - action: north
step 4 - action: west
step 5 - action: south
step 6 - action: south
step 7 - action: south
step 8 - action: east
step 9 - action: south
step 10 - action: suck
step 11 - action: north
step 12 - action: west
step 13 - action: suck
step 14 - action: north
step 15 - action: west
step 16 - action: west
step 17 - action: suck
step 18 - action: east
step 19 - action: north


5

# Tasks

_Submission Instructions:_ Use this notebook to prepare your submission. Complete this section with your code and results. You can add additional Markdown blocks for your description, comments in the code and use mathplotlib to produce charts. 

_Note:_ Try to keep the code simple! In this course, we want to learn about the algorithms and we often do not need to use object-oriented design. 


## Task 1: Implement a simulation environment [2 Points]

The simple environment above is not very realistic. Your environment simulator needs to follow the PEAS description from above. It needs to:

* Initialize the environment by storing the state of each square (clean/dirty) and making some dirty.
* Keep track of the agent's position.
* Call the agent function repeatedly and provide the agent function with the sensor inputs.  
* React to the agent's actions. E.g, by removing dirt from a square or moving the agent around unless there is a wall in the way.
* Keep track of the performance measure. That is, track the agent's actions until all dirty squares are clean and count the number of actions it takes the agent to complete the task.

The easiest implementation for the environment is to hold an 2-dimensional array to represent if squares are clean or dirty and to call the agent function in a loop until all squares are clean or a predefined number of steps have been reached (i.e., the robot runs out of energy).

The simulation environment needs to work with the simple randomized agent program from above and then it can be used for your agent implementation in the tasks below.

In [5]:
import numpy as np

# Note : n = Room Size (n x n) | p = Probability of dirt within each square
def new_env(n,p):
    
    # Initialize simulation environment (0 = Clean, 1 = Dirty)
    environment = np.random.choice(a=[0,1], size=(n, n+2), p=[1-p, p])
    
    # Bottom Wall
    environment = np.append(environment, np.full((1,n+2), "X"), axis=0)
    
    # Top Wall
    environment[:][0] = "X"
    
    # Left Wall
    environment[:,0] = "X"
    
    environment[:,n+1] = "X"
    
    
    return environment

In [6]:
def simulation_env(agent, max_steps, env, verbose=True):
    num_cleaned = 0
    
    # Initialize agent in NW corner
    x_axis, y_axis = 1,1
    
    # Total number of dirty squares
    dirty_squares = np.count_nonzero(env == 1)
    
    num_rows, num_cols = env.shape
    n = num_rows
    n_steps = 0
    for i in range(max_steps):
        x = x_axis
        y = y_axis
        dirty = True if(env[x_axis][y_axis]) else False
        n_bump, w_bump, e_bump, s_bump = False, False, False, False
        
        # Corner Boundaries
        if x_axis == 0 and y_axis == 0: n_bump, w_bump = True, True
        elif x_axis == 0 and y_axis == n - 1: n_bump, e_bump = True, True
        elif x_axis == n - 1 and y_axis == 0: s_bump, w_bump = True, True
        elif x_axis == n - 1 and y_axis == n - 1: s_bump, e_bump = True, True
            
        # Edge Boundaries
        elif x_axis == 0: n_bump = True
        elif y_axis == 0: w_bump = True
        elif y_axis == n - 1: e_bump = True
        elif x_axis == n - 1: s_bump = True
        
        # Set status of each bumper sensor
        bumpers = {"north" : n_bump, "south" : s_bump, "west" : w_bump, "east" : e_bump}
        
        action = agent(bumpers, dirty)
        
        location = "Moving! [" + str(x_axis) + "," + str(y_axis) + "]"
        if action == "north" and n_bump == False: x_axis -= 1
        if action == "west"  and w_bump == False: y_axis -= 1
        if action == "east"  and e_bump == False: y_axis += 1
        if action == "south" and s_bump == False: x_axis += 1
        if x != x_axis or y_axis != y: location += " --> [" + str(x_axis) + "," + str(y_axis) + "]"
        else: location = "Wall is in the way!"
        
        if (verbose and action != "suck"): print("step", i , "- action:", action, "  \t", location) 
        
        if (action == "suck"):
            if dirty != False:
                if verbose: print("step", i , "- action:", action, "  \t ---> Cleaning! " + str(dirty_squares - 1) + " dirty squares remaining")
                dirty_squares -= 1
                env[x_axis][y_axis] = 0
                num_cleaned = num_cleaned + 1
                if dirty_squares == 0:
                    if verbose: print("CLEANING COMPLETE!")
                    return i
                    #return num_cleaned
            elif verbose: print("step", i , "- action:", action, "  \t", "Already Clean!")
        n_steps = i
        
    return i  
    # return num_cleaned

In [7]:
my_env = new_env(n=5, p=0.2)

my_env

array([['X', 'X', 'X', 'X', 'X', 'X', 'X'],
       ['X', '1', '0', '0', '0', '0', 'X'],
       ['X', '1', '0', '0', '0', '0', 'X'],
       ['X', '0', '1', '0', '1', '0', 'X'],
       ['X', '0', '1', '0', '0', '0', 'X'],
       ['X', 'X', 'X', 'X', 'X', 'X', 'X']], dtype='<U21')

In [8]:
simulation_env(simple_randomized_agent, max_steps=100, env=my_env, verbose=True)

step 0 - action: north   	 Wall is in the way!
step 1 - action: east   	 Moving! [0,0] --> [0,1]
step 2 - action: south   	 Moving! [0,1] --> [1,1]
step 3 - action: suck   	 ---> Cleaning! -1 dirty squares remaining
step 4 - action: suck   	 ---> Cleaning! -2 dirty squares remaining
step 5 - action: south   	 Moving! [1,1] --> [2,1]
step 6 - action: south   	 Moving! [2,1] --> [3,1]
step 7 - action: east   	 Moving! [3,1] --> [3,2]
step 8 - action: east   	 Moving! [3,2] --> [3,3]
step 9 - action: north   	 Moving! [3,3] --> [2,3]
step 10 - action: north   	 Moving! [2,3] --> [1,3]
step 11 - action: suck   	 ---> Cleaning! -3 dirty squares remaining
step 12 - action: south   	 Moving! [1,3] --> [2,3]
step 13 - action: west   	 Moving! [2,3] --> [2,2]
step 14 - action: east   	 Moving! [2,2] --> [2,3]
step 15 - action: suck   	 ---> Cleaning! -4 dirty squares remaining
step 16 - action: south   	 Moving! [2,3] --> [3,3]
step 17 - action: north   	 Moving! [3,3] --> [2,3]
step 18 - actio

/home/blurry/anaconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


99

## Task 2:  Implement a simple reflex agent [1 Point] 

The simple reflex agent randomly walks around but reacts to the bumper sensor by not bumping into the wall and to dirt with sucking. Implement the agent program as a function.

_Note:_ The agent cannot directly use variable in the environment. It only gets the percepts as the arguments to the agent program function.

In [9]:
def simple_reflex_agent(agent, max_steps, env, verbose = True):
    num_cleaned = 0
    
    # Initialize agent in NW corner
    x_axis, y_axis = 0,0
    
    # Total number of dirty squares
    dirty_squares = np.count_nonzero(env == 1)
    
    num_rows, num_cols = env.shape
    n = num_rows
    
    n_steps = 0
    
    for i in range(max_steps):
        x = x_axis
        y = y_axis
        dirty = True if(env[x_axis][y_axis]) else False
        n_bump, w_bump, e_bump, s_bump = False, False, False, False
        
        # Corner Boundaries
        if x_axis == 0 and y_axis == 0: n_bump, w_bump = True, True
        elif x_axis == 0 and y_axis == n - 1: n_bump, e_bump = True, True
        elif x_axis == n - 1 and y_axis == 0: s_bump, w_bump = True, True
        elif x_axis == n - 1 and y_axis == n - 1: s_bump, e_bump = True, True
            
        # Edge Boundaries
        elif x_axis == 0: n_bump = True
        elif y_axis == 0: w_bump = True
        elif x_axis == n - 1: s_bump = True
        elif y_axis == n - 1: e_bump = True
        
        bumpers = {"north" : n_bump, "south" : s_bump, "west" : w_bump, "east" : e_bump}
        
        flag, action = True, True
        while(flag):
            action = agent(bumpers, dirty)
            if action == "suck": continue
            if bumpers[action] != True: flag = False
        
        output = "Moving! [" + str(x_axis) + "," + str(y_axis) + "]"
        if action == "north" and n_bump == False: x_axis -= 1
        if action == "west"  and w_bump == False: y_axis -= 1
        if action == "east"  and e_bump == False: y_axis += 1
        if action == "south" and s_bump == False: x_axis += 1
        if x != x_axis or y_axis != y:
            output += " --> [" + str(x_axis) + "," + str(y_axis) + "]"
            if dirty != False:
                output += " ---> Square is dirty. Cleaning! " + str(dirty_squares - 1) + " dirty squares remaining!"
                dirty_squares -= 1
                env[x_axis][y_axis] = 0
                num_cleaned = num_cleaned + 1
                
                if dirty_squares == 0:
                    if verbose: print("CLEANING COMPLETE!")
                    
                    return i
                    # return num_cleaned
        
                
        if (verbose): print("step", i , "- action:", action, "  \t", output) 
        n_steps = i 
    return n_steps

## Task 3: Implement a model-based reflex agent [3 Point]

This agent keeps track of the location and remembers where it has cleaned. Assume the agent knows how many squares the room has. It can move to a corner to determine its location and then is able to use more advanced navigation.

Describe how you define the __agent state__ and how your agent works before implementing it. _Note on implementing the state in Python:_ [Examples](https://github.com/luisegarduno/Ai-Projects/blob/master/Python_Code_Examples/store_agent_state_information.ipynb)

In [10]:
# Your short description of the state and your implementation goes here

In [11]:
# Your code goes here

class Agent:
    def __init__(self, initial_position=(0,0), name="Model-Based Reflex Agent"):
        self.position = initial_position
        self.name = name
        
    def act(self, percept_blocked):
        print("    ", self.name, ": My percept_blocked is", percept_blocked, "and I'm now in position", self.position)
        
        if percept_blocked: return ("stay")
        
        self.position = self.position + 1
        
        print("    ", self.name, ": I advance to position", self.position)
        
        return ("advance")

## Task 4: Simulation study [3 Points]

Compare the performance (the performance measure is defined in the PEAS description above) of the agents using  environments of different size. E.g., $5 \times 5$, $10 \times 10$ and
$100 \times 100$. Use at least 100 random runs for each. Present the results in a suitable format (tables, graphs) and discuss the differences.     
Here is some help with [charts and tables.](https://github.com/luisegarduno/Ai-Projects/blob/master/Python_Code_Examples/charts_and_tables.ipynb)

In [12]:
# Your code goes here

# The probability of there being dirt within each square
# still remains at 20%
p = 0.20

env_5x5 = new_env(n=5, p=0.2)
env_10x10 = new_env(n=10, p=0.2)
env_100x100 = new_env(n=100, p=0.2)

aye = simulation_env(simple_randomized_agent, max_steps=100, env=env_5x5, verbose=False)
aye2 = simple_reflex_agent(simple_randomized_agent, max_steps=100, env=env_5x5, verbose=False)

ayo = "aye {" + str(aye) + "}"
ayo2 = "ayo {" + str(aye2) + "}"
print(ayo + '\t' + ayo2)





aye {99}	ayo {99}


/home/blurry/anaconda3/envs/ML/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


In [13]:
avg = 0
for i in range(10):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-13-6484d141c8ed>, line 3)

Fill out the following table with the average performance measure for 100 random runs (you may also create this table with code):

|Size | Randomized Agent | Simple Reflex Agent | Model-based Reflex Agent |
|-----|------------------|---------------------|--------------------------|
|5x5 | | | |
|10x10 | | | |
|100x100 | | | |

In [ ]:
# Your discussion of the results goes here

## Task 5: Robustness of the agent implementations [1 Point] 

Describe how your agent implementations will perform 

* if it is put into a rectangular room with unknown size, 
* if the cleaning area can have an iregular shape (e.g., a hallway connecting two rooms), or 
* if the room contains obstacles (i.e., squares that it cannot pass through and trigger the bumper sensors).

In [ ]:
# Answer goes here

## Graduate student advanced task: Obstacles [1 Point]

__Undergraduate students:__ This is a bonus task you can attempt if you like [+1 Bonus point].

1. Change your simulation environment tor run experiments for the following problem: Add random obstacle squares that also trigger the bumper sensor. The agent does not know where the obstacles are. Observe how this changes the performance of the three implementations.

2. Describe what would need to be done to perform better with obstacles. Add code if you can. 

In [ ]:
# Your code and discussion goes here

## More advanced tasks to think about

You can think about these:

* __Unknown environment with obstacles:__ Implement an agent for an environment where the agent does not know how large the environment is (we assume it is rectangular), where it starts or where the obstacles are. An option would be to always move to the closest unchecked/uncleaned square.

* __Utility-based agent:__ Change the environment, so each square has a fixed probability of getting dirty again. We assume the agent has learned this information over time. For the implementation, we give this information to the agent as a 2-dimensional array of probabilities Cleaning one dirty square produces a utility of 1. Implement a utility-based agent that maximizes the expected utility over one full charge which lasts for 10000 time steps. This is very tricky!